In [1]:
import re
from datetime import datetime, timedelta,time
import calendar

import pickle
# load the model from disk
nlp = pickle.load(open('education nlp.pkl', 'rb'))

/home/elshan/Projects/Chatbot_pytorch/pytorch_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# opening the file in read mode
my_file = open("s.txt", "r")
  
# reading the file
data = my_file.read()
  
# replacing end splitting the text 
# when newline ('\n') is seen.
data_into_list = data.split("\n")
#print(data_into_list)
my_file.close()

In [4]:
def word_to_number(text):
    num_dict = {
        "zero": 0, "one": 1, "two": 2, "three": 3, "four": 4,
        "five": 5, "six": 6, "seven": 7, "eight": 8, "nine": 9,
        "ten": 10, "eleven": 11, "twelve": 12, "thirteen": 13, "fourteen": 14,
        "fifteen": 15, "sixteen": 16, "seventeen": 17, "eighteen": 18, "nineteen": 19,
        "twenty": 20, "thirty": 30, "forty": 40, "fifty": 50, "sixty": 60,
        "seventy": 70, "eighty": 80, "ninety": 90
    }
    
    words = text.lower().split()
    result = 0
    sub_total = 0
    for word in words:
        if word == "and":
            result += sub_total
            sub_total = 0
            continue
        num = num_dict.get(word, None)
        if num is not None:
            sub_total += num
        elif word == "hundred":
            sub_total *= 100
        elif word == "thousand":
            result += sub_total * 1000
            sub_total = 0
        else:
            return None # invalid input
    return result + sub_total

def parse_date(input_str):        
    now = datetime.now()
    date = None
    t = None
    # Match time formats using regex
    time_match = re.search(r'(?P<hour>\d{1,2})(?:[:|.](?P<minute>\d{2}))? ?(?P<am_pm>am|pm)?', input_str)
    if time_match is not None and sum(value is not None for value in time_match.groupdict().values()) >= 2:
        # Extract matched values from regex match
        hour = int(time_match.group('hour'))
        minute = int(time_match.group('minute') or 0)
        am_pm = time_match.group('am_pm')
        if hour > 12:
            pass
        # Convert 12-hour format to 24-hour format if necessary
        if am_pm and am_pm.lower() == 'pm' and hour != 12:
            hour += 12
        #print(hour)
        
        # Create time object
        if hour < 24:
            if minute:
                t = time(hour=hour, minute=minute)
            else:
                t = time(hour=hour)
                #print(t)
        
    # Parsing time
    time_regex = re.search(r'\b(\d+)\s*[hrs|hours|hr|h|hs]+\b', input_str, re.IGNORECASE)
    if time_regex:
        hr = int(time_regex.group(1))
        t = now + timedelta(hours=hr)
        date = t.date() if t.date() > now.date() else now.date()
        t = t.strftime('%H:%M:%S')
        #print(date, t)

    # Today
    if date is None and re.search(r'\btoday\b', input_str, re.IGNORECASE):
        date = now.date()
        #print(date)

    # Tomorrow
    if date is None and re.search(r'\btomorrow\b', input_str, re.IGNORECASE):
        date = (now + timedelta(days=1)).date()
        #print(date)

    # Day after tomorrow
    if date is None and re.search(r'\bthe day after tomorrow\b', input_str, re.IGNORECASE):
        date = (now + timedelta(days=2)).date()
        #print(date)

    # Weekday
    if date is None:
        weekdays = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
        for i, weekday in enumerate(weekdays):
            if re.search(r'\b' + weekday + r'\b', input_str, re.IGNORECASE):
                day = i - now.weekday()
                day += 7 if day <= 0 else 0
                date = (now + timedelta(days=day)).date()
                #print(date)
                break

    # Date with "in x days" format
    days_regex = re.search(r'\bin?\s*(.*)\s+days?\b', input_str, re.IGNORECASE)
    if date is None and days_regex:
        try:
            days = int(days_regex.group(1))
        except:
            days = days_regex.group(1)
            days=word_to_number(days)
        date = (now + timedelta(days=days)).date()
        #print(date)
    # Date with "in x weeks" format
    weeks_regex = re.search(r'\bin?\s*(.*)\s+weeks?\b', input_str, re.IGNORECASE)    
    if date is None and weeks_regex:
        try:
            weeks = int(weeks_regex.group(1))
            date = (now + timedelta(weeks=weeks)).date()
        except:
            weeks_str = weeks_regex.group(1)
            weeks = word_to_number(weeks_str)
        date = (now + timedelta(weeks=weeks)).date()
    
    # Date with "in x months" format
    months_regex = re.search(r'\bin?\s*(.*)\s+months?\b', input_str, re.IGNORECASE)
    if date is None and months_regex:
        try:
            months = int(months_regex.group(1))
        except:
            months = months_regex.group(1)
            months=word_to_number(months)


        year = now.year + (now.month + months - 1) // 12
        month = (now.month + months - 1) % 12 + 1
        day = min(now.day, calendar.monthrange(year, month)[1])
        date = datetime(year, month, day).date()
        #print(date)
    
    # Date with "in x years" format
    years_regex =re.search(r'\bin?\s*(.*)\s+years?\b', input_str, re.IGNORECASE)
    if date is None and years_regex:
        try:
            years = int(years_regex.group(1))
        except:
            years = years_regex.group(1)
            years=word_to_number(years)
        date = (now + timedelta(days=years*365)).date()
        #print(date)
    
    # Date with "next week" format
    if date is None and re.search(r'\bnext week\b', input_str, re.IGNORECASE):
        date = (now + timedelta(weeks=1)).date()
        #print(date)

    # Date with "next month" format
    if date is None and re.search(r'\bnext month\b', input_str, re.IGNORECASE):
        year = now.year + (now.month + 1) // 12
        month = (now.month + 1) % 12 
        day = min(now.day, calendar.monthrange(year, month)[1])
        date = datetime(year, month, day).date()
        #print(date)
    # Date with "next year" format
    if date is None and re.search(r'\bnext year\b', input_str, re.IGNORECASE):
        year = now.year + 1
        month = now.month
        day = min(now.day, calendar.monthrange(year, month)[1])
        date = datetime(year, month, day).date()
        #print(date)

    # Date with "april, 30", "april 30", "april30","30 Apr","30 apr", "30th April", "april 30th", "30th of the April" format
    date_pattern = re.compile(r'(?P<day1>\d*)[\s|,|th]*(?P<month>jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)[\s|,]*(?P<day2>\d*)', re.IGNORECASE)
    if date is None and re.search(date_pattern, input_str):
        match = date_pattern.search(input_str)
        month_str = match.group('month')
        if match.group('day1'):
            day_str = match.group('day1')            
        elif match.group('day2'):
            day_str = match.group('day2')
        else:
            day_str = None
        month_dict = {
            'jan': 'January','feb': 'February','mar': 'March','apr': 'April','may': 'May','jun': 'June','jul': 'July','aug': 'August','sep': 'September','oct': 'October','nov': 'November','dec': 'December'
        }

        if month_str.lower()[:3] in month_dict:
            month_str = month_dict[month_str.lower()[:3]]

        month = datetime.strptime(month_str, '%B').month
        if day_str is None:
            day_str = '1'
        date = datetime(now.year, month, int(day_str)).date()
        #print(date) 
    
    return date, t

In [7]:
#data_into_list =["In 2 days i want to take a walk from via Paisiello 26 to Viale Elvezia, 18"] 
def extract_entity(text):    
    from_location = None
    to_location = None
    event_date = None
    event_time = None
    transport = None
    doc = nlp(text)
    for ent in doc.ents:
        time_node = False
        #print(ent)
        
        if ent.label_ == "TRANSPORT":
            transport = ent
        if ent.label_ == "LOC-TO":
            to_location = ent
        if ent.label_ == "LOC-FROM":
            from_location = ent
        if ent.label_ == "DATE" :
            if time_node is True:
                event_date,_ = parse_date(str(ent.text))
            else:
                event_date,event_time = parse_date(str(ent.text))
        if ent.label_ == "TIME" and event_time is None:
            _,event_time = parse_date(str(ent.text))
            if event_time is not None:
                time_node = True
    return event_date,event_time,from_location,to_location,transport
for text in data_into_list:
    print("#"*50)
    print(text)
    event_date,event_time,from_location,to_location,transport = extract_entity(text)
    print()
    print("DATE: ", str(event_date)) 
    print("TIME: ",str(event_time)) 
    print("FROM: ",str(from_location)) 
    print("TO: ",str(to_location)) 
    print("TRANSPORT: ",str(transport))

#print(extract_entity(text)[1])

##################################################
Tomorrow at 3pm i want to take a taxi from via Paisiello 26 to Viale Elvezia, 18

DATE:  2023-03-31
TIME:  15:00:00
FROM:  via Paisiello 26
TO:  Viale Elvezia, 18
TRANSPORT:  taxi
##################################################
Tomorrow at 17:25 i want to take a shared car from via Paisiello 26 to Viale Elvezia, 18

DATE:  2023-03-31
TIME:  17:25:00
FROM:  via Paisiello 26
TO:  Viale Elvezia, 18
TRANSPORT:  shared car
##################################################
let's go tomorrow at 5am to centre

DATE:  2023-03-31
TIME:  05:00:00
FROM:  None
TO:  centre
TRANSPORT:  None
##################################################
i want to create a new event

DATE:  None
TIME:  None
FROM:  None
TO:  None
TRANSPORT:  None
##################################################
In 12 hours i want to take a shared car from via Paisiello 26 to Viale Elvezia, 18

DATE:  None
TIME:  21:58:25
FROM:  via Paisiello 26
TO:  Viale Elvezia, 18
TRANSPOR

In [233]:
import json
data_dict ={"text": None, "annotations":[]}

sentence = "In 2 days i want to take a shared scooter from via Paisiello 26 to Viale Elvezia, 18"
words = ["In 2 days","shared scooter","via Paisiello 26", "Viale Elvezia, 18"]
data_dict["text"] = sentence
for i,word in enumerate(words):
    d = {}
    d["label"] = ""
    d["start"] = sentence.index(word)
    d["end"] = sentence.index(word) + len(word)
    d["word"] = word
    data_dict["annotations"].append(d)

print(json.dumps(data_dict, ensure_ascii=False))


{"text": "In 2 days i want to take a shared scooter from via Paisiello 26 to Viale Elvezia, 18", "annotations": [{"label": "", "start": 0, "end": 9, "word": "In 2 days"}, {"label": "", "start": 27, "end": 41, "word": "shared scooter"}, {"label": "", "start": 47, "end": 63, "word": "via Paisiello 26"}, {"label": "", "start": 67, "end": 84, "word": "Viale Elvezia, 18"}]}


In [217]:
import json

ent_text_dict = {"DATE": [], "TIME": [], "LOC-TO": [], "LOC-FROM": [], "TRANSPORT": []}

for i in data_into_list:
    # print("#################################")
    # print(i)
    doc=nlp(i)
    for ent in doc.ents:
        # add the entity text to the corresponding list in the dictionary
        ent_text_dict[ent.label_].append(ent.text)
#print(ent_text_dict)

for key in ent_text_dict:
    ent_text_dict[key] = list(set([x.lower() for x in ent_text_dict[key]]))
#print(json.dumps(ent_text_dict, ensure_ascii=False))
print(json.dumps(ent_text_dict, ensure_ascii=False))

{"DATE": ["in two weeks", "in 23 hours", "in 24 hours", "in april", "18 june", "next month", "in 2 weeks", "25th april", "in 25th april", "april 25", "in 1hr", "april, 25", "25 april", "tomorrow", "friday", "next week", "18 june via torino 12", "tuesday", "in 3 days"], "TIME": ["3:05 pm", "5 am", "in 1h i", "15 pm", "in 12 hours", "in 12 hrs", "3.25 pm", "3:15 pm", "3:25 pm", "1pm", "in 12h i", "create", "in 12 hs", "3 pm", "3.5 pm", "18:00", "5am", "17:25", "in 2 days", "3.15 pm", "3:30 pm", "18:35", "13", "3pm"], "LOC-TO": ["123 main st, new york", "viale elvezia, 18", "123 main st, new york, ny", "create a new", "garibaldi", "via garibaldi, 4", "duomo", "centre", "last", "via torino 12", "london", "prague", "office", "mountains", "viale elvezia, 18, 20154 milano"], "LOC-FROM": ["123 main st, new york", "123 main st, new york, ny", "home", "duomo", "via ippodromo, 5", "via paisiello 26", "via ippodromo", "via cibrario, 40"], "TRANSPORT": ["taxi", "shared car"]}
